In [37]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
df = pd.read_csv('/content/drive/My Drive/Resources/kingcounty/seattle_sales_cleaned_data.csv')
# Review the DataFrame
df

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,year
0,0,1/2/2018,665000,47.56,-122.32,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
1,1,1/9/2018,499000,47.52,-122.32,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
2,2,2/14/2018,675000,47.62,-122.30,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018
3,3,2/13/2018,1150000,47.72,-122.34,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.50,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018
4,4,2/14/2018,705000,47.67,-122.32,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24933,24945,12/8/2022,895000,47.69,-122.33,SEATTLE,NR3,440000,327000,1926,3000,1100,7,7,3,1.50,3,2,0,0,370,0,0,0,0,0,0,0,0,0,0,0,0,0,2022
24934,24946,12/5/2022,1900000,47.60,-122.29,SEATTLE,NR3,790000,1432000,1924,5016,3000,10,8,3,2.00,4,2,1,0,240,0,0,0,0,0,0,0,2,2,0,0,1,0,2022
24935,24947,12/17/2022,1750000,47.63,-122.31,SEATTLE,NR3,853000,727000,1908,4800,2900,9,8,3,1.50,4,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2022
24936,24948,12/15/2022,1250000,47.68,-122.39,SEATTLE,NR3,497000,532000,1928,3825,1730,8,8,3,1.00,3,1,1,0,180,0,0,0,0,0,0,0,0,0,0,0,0,0,2022


In [57]:
df['sale_date'] = pd.to_datetime(df['sale_date'])
df['sale_month'] = df['sale_date'].dt.month
df['sale_day'] = df['sale_date'].dt.day
df.rename(columns={'year': 'sale_year'}, inplace=True)
df.head()

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,0,2018-01-02,665000,47.56,-122.32,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,2
1,1,2018-01-09,499000,47.52,-122.32,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,9
2,2,2018-02-14,675000,47.62,-122.30,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2,14
3,3,2018-02-13,1150000,47.72,-122.34,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.50,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018,2,13
4,4,2018-02-14,705000,47.67,-122.32,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,14


In [58]:
df['garb_sqft'].value_counts()

0       18360
200      1084
240       607
220       405
180       339
        ...  
1170        1
950         1
115         1
1370        1
255         1
Name: garb_sqft, Length: 119, dtype: int64

In [59]:
df['gara_sqft'].value_counts()

0       20197
240       302
400       246
440       232
220       227
        ...  
1480        1
900         1
1180        1
880         1
1050        1
Name: gara_sqft, Length: 123, dtype: int64

In [60]:
df['wfnt'].value_counts()

0    24775
6       99
3       48
8       11
5        3
4        2
Name: wfnt, dtype: int64

In [61]:
df = df.drop(columns=['wfnt'])
df = df.drop(columns=['garb_sqft'])
df = df.drop(columns=['gara_sqft'])
df = df.drop(columns=['id'])

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24938 entries, 0 to 24937
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   sale_date         24938 non-null  datetime64[ns]
 1   sale_price        24938 non-null  int64         
 2   latitude          24938 non-null  float64       
 3   longitude         24938 non-null  float64       
 4   city              24938 non-null  object        
 5   zoning            24938 non-null  object        
 6   land_val          24938 non-null  int64         
 7   imp_val           24938 non-null  int64         
 8   year_built        24938 non-null  int64         
 9   sqft_lot          24938 non-null  int64         
 10  sqft              24938 non-null  int64         
 11  grade             24938 non-null  int64         
 12  fbsmt_grade       24938 non-null  int64         
 13  home_condition    24938 non-null  int64         
 14  stories           2493

In [63]:
df['zoning'].value_counts()
category_mapping = {
    'NR3': 1,
    'NR2': 2,
    'SF 5000': 3,
    'RSL (M)': 4,
    'SF 7200': 5,
    'NR1': 6,
    'SF 9600': 7
}

# Replace categorical values with integer values using the mapping
df['zoning'] = df['zoning'].map(category_mapping)
df['zoning'].value_counts()

1    16716
2     4014
3     2191
4     1293
5      360
6      313
7       51
Name: zoning, dtype: int64

In [64]:
df['sale_date'] = pd.to_datetime(df['sale_date'])
df['sale_date'] = df['sale_date'].astype('int64') // 10**9
df['sale_date']

0        1514851200
1        1515456000
2        1518566400
3        1518480000
4        1518566400
            ...    
24933    1670457600
24934    1670198400
24935    1671235200
24936    1671062400
24937    1671148800
Name: sale_date, Length: 24938, dtype: int64

In [65]:
df = df.drop(columns=['city'])

In [66]:
df = df.drop(columns=['sale_date'])

In [67]:
test_df = df[df['sale_price'] == 605000]
test_df

,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
186,605000,47.53,-122.32,3,32000,69000,1950,9375,1020,6,5,3,1.00,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2020,10,6
1845,605000,47.73,-122.34,3,68000,121000,1948,5250,1710,6,0,5,2.00,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2019,3,28
2011,605000,47.51,-122.25,3,74000,137000,1908,7910,2740,7,7,3,2.00,3,2,0,0,0,0,0,0,0,0,2,0,0,2,0,2019,11,7
2187,605000,47.52,-122.26,3,66000,116000,1903,5250,1830,7,0,3,2.00,4,1,1,0,0,0,0,0,0,0,2,0,0,2,0,2020,10,16
2213,605000,47.54,-122.32,3,21000,91000,1904,4000,920,7,0,4,1.00,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2020,11,4
2266,605000,47.53,-122.28,3,37000,97000,1925,6088,1140,7,0,3,1.00,4,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2021,2,18
2652,605000,47.56,-122.30,1,264000,477000,1971,5319,1920,7,7,3,1.00,5,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,3
4270,605000,47.70,-122.34,1,475000,375000,1934,4824,1840,7,7,4,1.50,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,6,28
4556,605000,47.51,-122.27,2,182000,475000,1958,6837,2400,7,6,4,1.00,6,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,7,19
4597,605000,47.51,-122.38,1,274000,210000,1937,5320,670,6,0,4,1.00,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,7,24


In [75]:
# Specify the column to exclude from the subset
column_to_exclude = 'sale_day'

# Create a subset of columns by excluding the specified column
subset_columns = [col for col in df.columns if col != column_to_exclude]

# Create a new DataFrame with the subset of columns
df_subset = df[subset_columns]

# Remove duplicate rows based on the subset of columns
df_no_duplicates = df_subset.drop_duplicates()
df = df_no_duplicates

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24818 entries, 0 to 24937
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sale_price        24818 non-null  int64  
 1   latitude          24818 non-null  float64
 2   longitude         24818 non-null  float64
 3   zoning            24818 non-null  int64  
 4   land_val          24818 non-null  int64  
 5   imp_val           24818 non-null  int64  
 6   year_built        24818 non-null  int64  
 7   sqft_lot          24818 non-null  int64  
 8   sqft              24818 non-null  int64  
 9   grade             24818 non-null  int64  
 10  fbsmt_grade       24818 non-null  int64  
 11  home_condition    24818 non-null  int64  
 12  stories           24818 non-null  float64
 13  beds              24818 non-null  int64  
 14  bath_full         24818 non-null  int64  
 15  bath_3qtr         24818 non-null  int64  
 16  bath_half         24818 non-null  int64 

In [77]:
summary_stats = df.describe()
print(summary_stats)

       sale_price  latitude  longitude   zoning   land_val    imp_val  \
count    24818.00  24818.00   24818.00 24818.00   24818.00   24818.00   
mean   1062638.23     47.63    -122.34     1.62  487062.73  487121.52   
std     654882.86      0.07       0.04     1.10  344179.71  347188.46   
min      65000.00     47.50    -122.42     1.00       0.00       0.00   
25%     698000.00     47.56    -122.38     1.00  291000.00  279000.00   
50%     889999.50     47.64    -122.34     1.00  432000.00  427000.00   
75%    1250000.00     47.68    -122.30     2.00  605000.00  606000.00   
max   14850000.00     47.73    -122.24     7.00 7560000.00 7946000.00   

       year_built  sqft_lot     sqft    grade  fbsmt_grade  home_condition  \
count    24818.00  24818.00 24818.00 24818.00     24818.00        24818.00   
mean      1942.57   6002.42  1901.74     7.32         4.09            3.73   
std         27.57   3649.74   852.44     1.04         3.41            0.80   
min       1900.00    773.00   

In [78]:
columns = ['sale_price', 'sqft_lot', 'sqft', 'land_val', 'imp_val']

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Calculate IQR for the specified columns
Q1 = df[columns].quantile(0.25)
Q3 = df[columns].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for the specified columns
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers for the specified columns
filtered_df = df[~((df[columns] < lower_bound) | (df[columns] > upper_bound)).any(axis=1)]
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.head()

,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month
0,665000,47.56,-122.32,3,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
1,499000,47.52,-122.32,3,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
2,675000,47.62,-122.30,3,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2
3,705000,47.67,-122.32,3,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2
4,650000,47.67,-122.28,3,93000,100000,1948,6750,830,7,0,4,1.00,2,1,0,0,0,0,2,0,0,0,0,0,0,0,0,2018,2


In [79]:
y = filtered_df["sale_price"].ravel()
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).ravel()
X = filtered_df.copy()
X.drop("sale_price", axis=1, inplace=True)

In [80]:
y_scaled[:5]

array([-0.73340732, -1.21938161, -0.70413176, -0.61630508, -0.77732066])

In [81]:
X.head()

,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month
0,47.56,-122.32,3,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
1,47.52,-122.32,3,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
2,47.62,-122.30,3,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2
3,47.67,-122.32,3,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2
4,47.67,-122.28,3,93000,100000,1948,6750,830,7,0,4,1.00,2,1,0,0,0,0,2,0,0,0,0,0,0,0,0,2018,2


In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

In [83]:
regression_model = LinearRegression()

# Fit the model using training data
regression_model.fit(X_train, y_train)

# Predict values using the trained model
y_pred = regression_model.predict(X_test)

In [84]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.46627050125358416
R-squared: 0.5265198104838655


In [85]:
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [86]:
y_pred2 = rf_model.predict(X_test)

In [87]:
pd.options.display.float_format = '{:.2f}'.format
y_pred2_original = scaler.inverse_transform(y_pred2.reshape(-1, 1)).ravel()

# Inverse transform the scaled actual prices to get original prices
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

mse = mean_squared_error(y_test_original, y_pred2_original)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_original, y_pred2_original)
rmse_rounded = round(rmse) # Round and convert to integer

absolute_diff = np.abs(y_test_original - y_pred2_original)

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual Price': y_test_original,
    'Predicted Price': y_pred2_original,
    'Absolute Difference': absolute_diff
})

# Print the comparison DataFrame
print(comparison_df)
print("Root Mean Squared Error:", rmse_rounded)
print("R-squared:", r2)

      Actual Price  Predicted Price  Absolute Difference
0        859900.00        909716.38             49816.38
1        915000.00       1289424.76            374424.76
2        885000.00        857975.78             27024.22
3        850000.00        799591.94             50408.06
4        475000.00        870958.47            395958.47
...            ...              ...                  ...
4346     819500.00        927134.62            107634.62
4347    1250000.00       1142971.25            107028.75
4348     835000.00        779016.08             55983.92
4349     885000.00        753672.75            131327.25
4350    1688750.00       1574715.94            114034.06

[4351 rows x 3 columns]
Root Mean Squared Error: 166193
R-squared: 0.7596202163804575
